In [3]:
import numpy as np

# ExaFMM-T
import exafmm.laplace as laplace

# PyExaFMM
from fmm import Fmm
from fmm.kernel import laplace_p2p_serial, laplace_gradient
from fmm.surface import scale_surface

# Adaptoctree
import adaptoctree.morton as morton
import adaptoctree.tree as tree

In [10]:
# ! rm test.hdf5 && fmm generate-test-data -c test && fmm compute-operators -c test

In [11]:
# ! rm test_sphere.hdf5 && fmm generate-test-data -c test_sphere && fmm compute-operators -c test_sphere

In [9]:
# Compile test experiment to cache numba functions
e = Fmm('test')
e.run()
laplace_p2p_serial(e.sources, e.targets, e.source_densities)
laplace_gradient(e.sources, e.targets, e.source_densities)

array([[ 15.34029  ,   2.0270848,  71.88409  ],
       [ 52.69213  ,  58.177074 ,  38.692673 ],
       [ 86.45617  ,  23.862509 ,  68.68593  ],
       ...,
       [-31.813215 , -64.628845 , -57.76195  ],
       [-49.487106 , -21.039127 , -26.436289 ],
       [-49.536842 ,   4.2401443,  -0.6216814]], dtype=float32)

In [12]:
e = Fmm('test_sphere')

In [13]:
e.nleaves

17017

In [14]:
# direct = laplace_p2p_serial(
#     e.sources,
#     e.targets,
#     e.source_densities
# )

In [19]:
! fmm generate-test-data -c C4E4 && fmm compute-operators -c C4E4
! fmm generate-test-data -c C5E5 && fmm compute-operators -c C5E5
! fmm generate-test-data -c C6E6 && fmm compute-operators -c C6E6
! fmm generate-test-data -c C7E7 && fmm compute-operators -c C7E7

Generating sphere sources & targets
Computing operators
Computing octree
Computing Inner Surface of Order 4
Computing Outer Surface of Order 4
Computing Inverse of Check To Equivalent Gram Matrix
Computing M2M & L2L Operators
Computed (1/8) M2M/L2L operators
Computed (2/8) M2M/L2L operators
Computed (3/8) M2M/L2L operators
Computed (4/8) M2M/L2L operators
Computed (5/8) M2M/L2L operators
Computed (6/8) M2M/L2L operators
Computed (7/8) M2M/L2L operators
Computed (8/8) M2M/L2L operators
Computed operators for (1/5) M2L Levels
Computed operators for (2/5) M2L Levels
Computed operators for (3/5) M2L Levels
Computed operators for (4/5) M2L Levels
Computed operators for (5/5) M2L Levels
Total time elapsed 0 minutes and 8 seconds
Generating sphere sources & targets
Computing operators
Computing octree
Computing Inner Surface of Order 5
Computing Outer Surface of Order 5
Computing Inverse of Check To Equivalent Gram Matrix
Computing M2M & L2L Operators
Computed (1/8) M2M/L2L operators
Computed

In [20]:
# Discretisation order of check surface
Cvec = [4, 5, 6, 7]

# Discretisation order of equivalent surface
Evec = [4, 5, 6, 7]

# Load experimental data into PyExaFMM
pyfmmvec = [Fmm(f'C{C}E{C}') for C in Cvec]

In [21]:
# Load experimental data into ExaFMM-T
exafmmvec = []
exafmmtreevec = []

for e in pyfmmvec:
    # create a list of source instances
    sources = laplace.init_sources(e.sources, e.source_densities)

    # create a list of target instances
    targets = laplace.init_targets(e.targets)
    
    # Expansion order
    p = e.config['order_equivalent']
    fmm = laplace.LaplaceFmm(p=p, ncrit=e.config['max_points'], filename=f'C{p}E{p}.dat')
    exafmmvec.append(fmm)
  
    tree = laplace.setup(sources, targets, fmm)
    exafmmtreevec.append(tree)

In [22]:
# Evaluate PyExaFMM experiments
for e in pyfmmvec:
    e.run()

# Evaluate ExaFMM-T experiments
trg_values = []
for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
    trg_values.append(laplace.evaluate(tree, fmm))

In [23]:
pyerrvec = []
exerrvec = []

for i in range(len(pyfmmvec)):
    pyerrvec.append(np.mean(abs(pyfmmvec[i].target_potentials[:,0]-direct)/direct))
    exerrvec.append(np.mean(abs(trg_values[i][:,0]-direct)/direct))

In [24]:
pyerrvec

[0.00010006761, 4.547916e-06, 2.413275e-06, 1.9974611e-06]

In [25]:
exerrvec

[8.747951e-05, 4.1538833e-06, 1.665213e-06, 2.8606285e-06]

In [26]:
1e-4

0.0001